In [ ]:
# Loading data

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
DF=pd.concat([train,test])
pd.options.display.max_columns=30

In [ ]:
# Decoding names
def decodeTitle(name):
    signif_titles=['Mr.','Miss.','Mrs.','Master.'] 
    # cut exactly title from column Name
    title=name[name.find(',')+2:name.find('.')+1]
    if title in signif_titles:  return(title)
    else: return('Arist.')
    
DF['Title']=DF['Name'].apply(decodeTitle)
DF.loc[88,'Title']='Miss'
DF.loc[550,'Name']='Thayer, Mr. NOjo Borland Jr'

In [ ]:
# Decodng cabins
def number(string):
    if pd.notnull(string):
        numbers=[int(i) for i in re.findall('(\d+)',string)]
        return(np.mean(numbers))
def symbol(string):
    if pd.notnull(string):
        return(True)
    else: return(False)

DF['CabinSymbol'] = DF.Cabin.apply(symbol)
DF['CabinNumber'] = DF.Cabin.apply(number)

In [ ]:
Ages, Fares=[0],[0]
for i in [1,2,3]:
    Ages.append(int(DF[DF.Pclass==i]['Age'].mean()))
    Fares.append(int(DF[DF.Pclass==i]['Fare'].mean()))
def funca(row):
    if row[0]>0: 
        age=row[0]
    else: 
        age=Ages[row[7]]
    return(age)
def funcb(row):
    if row[3]>0: 
        fare=row[3]
    else: 
        fare=Fares[row[7]]
    return(fare)
                 
DF['Age']=DF.apply(funca,axis=1)
DF['Fare']=DF.apply(funcb,axis=1)

In [ ]:
def name(name):
        name=name[name.find('. ')+2:]
        if name.find(' ')==-1:
            return(name)
        else: return(name[:name.find(' ')])
def virgin_surname(name):
    if name.find('(')!=-1:
        return(name[name.rfind(' '):name.find(')')])
def husb_or_wife(row):
    if (row[8]=='female') and (row[4].find('Mrs.')!=-1):
        return(row[17])
DF['Surname']=DF.Name.apply(lambda x: x[:x.find(',')])
DF['Virg_surname']=DF.Name.apply(virgin_surname)
DF['First_name']=DF.Name.apply(name)
DF['Husb_name']=DF.apply(husb_or_wife,axis=1)
Family=pd.DataFrame(DF.Surname.value_counts())

In [ ]:
# DF['wife']=DF.apply(find_my_wife, axis=1)
# DF['how_is_wife']=DF.apply(how_is_wife, axis=1)
# DF['childs']=DF.apply(find_my_childs, axis=1)
# DF['bro']=DF.apply(find_my_bro, axis=1)
# DF['how_are_childs']=DF.apply(how_are_childs, axis=1)    
# DF['how_are_bro']=DF.apply(how_are_bro,axis=1)
DF['parents']=DF.apply(find_my_parents, axis=1)
DF['how_are_parents']=DF.apply(how_are_parents, axis=1) 
DF['husband']=DF.apply(find_my_husb, axis=1)
DF['how_is_my_husb']=DF.apply(how_is_husb, axis=1)

In [ ]:
DF['Title']=DF['Title'].astype('category').cat.codes
DF['CabinSymbol']=DF['CabinSymbol'].astype('category').cat.codes
DF['Sex']=DF['Sex'].astype('category').cat.codes
DF['Embarked']=DF['Embarked'].astype('category').cat.codes

In [ ]:
from sklearn.utils import shuffle
DF = shuffle(DF, random_state = 187)

In [ ]:
DF_train=DF[DF.Survived.notnull()]
DF_test=DF[DF.Survived.notnull()==False]
X_train=DF_train[['How_is_my_wife','Pclass','Sex','Age','Cabin_Symbol','nearest','']]
X_train.fillna(-1, inplace=True)
Y_train=DF_train['Survived']
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
optimizer = GridSearchCV(RandomForestClassifier(), \
                         {'max_features' : [0.4, 0.6, 0.8, 1.0], 
                          'max_depth' : [5, 10, 15, 25, 35, 45, 55, 65], 
                          'n_estimators' : [1, 5, 10, 15, 30, 50, 70, 90, 100]},\
                         cv = 15, \
                         n_jobs=-1)
optimizer.fit(X_train,Y_train)
# print(cross_val_score(optimizer,DF_train[['How_is_my_siblings','Pclass','Title']], DF_train['Survived']))
print('Random Forest Best Result', optimizer.best_score_)
print('Best Parameters', optimizer.best_params_)

In [ ]:
X_test=DF_test[['How_is_my_wife','how_is_my_husb','how_are_childs','how_are_bro','Pclass','Sex','Title']]
X_test.fillna(-1, inplace=True)
def writeAnswer(filename, optimizer):
    prediction = optimizer.predict(X_test)
    submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.to_csv(filename, index=False)
    
writeAnswer('Its_alive.csv', optimizer)

In [ ]:
DF_train=new[new.Survived.notnull()]
# DF_train=DF_train[DF['count']>3]
X_train=DF_train[['How_is_my_wife','how_is_my_husb','how_are_childs','how_are_bro']]
Y_train=DF_train['Survived']
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print(cross_val_score(model,X_train, Y_train))


In [ ]:
## Big pack of setting of relationship between passengers
def find_my_wife(row):
    if (row[8]=='male') and (row[12]!='Master.') and ('Jr' not in row[4]):
        probably_wife=DF[(DF['Surname']==row[15]) & (DF['Husb_name']==row[17]) & (DF['Title']=='Mrs.') & (DF['Title']!='Arist.')]
        if len(probably_wife)!=0:
            return(int(probably_wife['PassengerId']))
        else:return(-999)
    else: return(-888)

#0 - not survived, 1- survived, 2 - not in a board, 3 - woman or child, 4 - no data
def how_is_wife(row):
    if row[19]>0:
        surv=DF[DF['PassengerId']==row[19]]['Survived']
        if pd.notnull(surv.values):
            return(surv.values[0])
        else: return(4)
    elif row[19]>-900: return(3)
    else: return(2)


def find_my_childs(row):
    if row[12] in ['Mrs.','Mr.','Arist.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Master.') | (DF['Title']=='Miss.') | (DF['Title']=='Mr.'))]
        probably_childs=probably_childs[(probably_childs['SibSp']==row[5]-1) & (probably_childs['Name']!=row[4])]
        probably_childs=probably_childs[(probably_childs['wife'].notnull()==False) | (probably_childs['SibSp']!=1)]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def find_my_bro(row):
    if row[12] in ['Master.','Miss.','Mr.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Master.') | (DF['Title']=='Miss.') | (DF['Title']=='Mr.'))]
        probably_childs=probably_childs[(probably_childs['Parch']==row[5]) & (probably_childs['SibSp']==row[9]) & (probably_childs['Name']!=row[4])]
        probably_childs=probably_childs[probably_childs['PassengerId'].isin(row[21])==False]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def how_are_childs(row):
    if row[21][0]>0:
        surv_index,j=[],np.nan
        for i in row[21]:
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
            j=np.nanmean(surv_index)
        if pd.notnull(j):
            return(j)
        else:return(4)
    elif row[21][0]>-900: return(3)
    else: return(2)   
    
    
def how_are_bro(row):
    if row[22][0]>0:
        surv_index,j=[],np.nan
        for i in row[22]:
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
            j=np.nanmean(surv_index)
        if pd.notnull(j):
            return(j)
        else:return(4)
    elif row[22][0]>-900: return(3)
    else: return(2)   
    

def find_my_parents(row):
    if row[12] in ['Miss.','Master.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Mrs.') | (DF['Title']=='Mr.') | (DF['Title']=='Arist.'))]
        probably_childs=probably_childs[(probably_childs['SibSp']==row[5]-1) & (probably_childs['Name']!=row[4])]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def how_are_parents(row):
    if row[25][0]>0:
        surv_index,j=[],np.nan
        for i in row[25]:
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
            j=np.nanmean(surv_index)
            if pd.notnull(j):
                return(j)
            else:return(4)
    elif row[25][0]>-900: return(3)
    else: return(2)  


def find_my_husb(row):
    if (row[8]=='female') and (row[12]!='Miss.'):
        probably_wife=DF[(DF['Surname']==row[15]) & (DF['Title']!='Master.') & (DF['First_name']==row[17]) & (DF['Name']!=row[4])]
        if len(probably_wife)!=0:
            return(int(probably_wife['PassengerId']))
        else:return(-999)
    else: return(-888)

def how_is_husb(row):
    if row[27]>0:
        surv=DF[DF['PassengerId']==row[27]]['Survived']
        if pd.notnull(surv.values):
            return(surv.values[0])
        else: return(4)
    elif row[27]>-900: return(3)
    else: return(2)

In [ ]:
pd.crosstab(DF.how_are_bro,DF.how_are_parents)

In [ ]:
pd.crosstab(DF.how_is_wife,DF.Survived)

In [ ]:
a=[np.nan,np.nan,np.nan]
np.nanmean(a)

In [ ]:
pd.crosstab(DF.Survived,DF.how_are_childs)

In [ ]:
DF[DF['PassengerId']==408]['Survived'].values[0]

In [ ]:
[-99,0]